In [5]:
img_path = '/content/drive/MyDrive/стажировка/12_classes_final/module_detectron2/test_examples/'
result_dir = '/content/drive/MyDrive/стажировка/12_classes_final/module_detectron2/results/'

module_dir = '/content/drive/MyDrive/стажировка/12_classes_final/module_detectron2/'

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install detectron2 -r /content/drive/MyDrive/стажировка/12_classes_final/module_detectron2/requirements.txt

Looking in links: https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.7/index.html
     |████████████████████████████████| 6.8MB 2.3MB/s 
     |████████████████████████████████| 2.2MB 5.6MB/s 
     |████████████████████████████████| 276kB 27.2MB/s 
  Created wheel for fvcore: filename=fvcore-0.1.2.post20210115-cp36-none-any.whl size=40892 sha256=236df28ddc56d058e4af17e4c2e83430194de79a2d088c242f981d9555f8e181
  Stored in directory: /root/.cache/pip/wheels/cc/0f/bc/fb742771f4a877f1211dd6ed36283b0db9c0ceb2b409b4a039
  Created wheel for pyyaml: filename=PyYAML-5.3.1-cp36-cp36m-linux_x86_64.whl size=44621 sha256=7dc6ccecce0189bb7c0e817f7f8f09fe6f694ef669382aad06405f3ee401bcf5
  Stored in directory: /root/.cache/pip/wheels/a7/c1/ea/cf5bd31012e735dc1dfea3131a2d5eae7978b251083d6247bd
  Created wheel for iopath: filename=iopath-0.1.2-cp36-none-any.whl size=10508 sha256=ed1f9cee130aff1c51874516f84734a9a5d6f59f8e69859b9af72f2ae6a683e1
  Stored in directory: /root/.cache/pip/wheels/9e/0

In [3]:
def predict(img_path, module_dir, result_dir):

  '''
  Функция предикта карты сегментации стройки для 13 классов
  Входные параметры:
    - img_path - путь к изображению
    - module_dir - путь к папке с модулем
    - result_dir - путь к папке, куда сохранять предикт
  Выходные параметры:
    - pred_path - путь, куда сохранено изображение
  '''

  # Импорт модулей
  import torch, torchvision
  import detectron2
  import numpy as np
  import os, json, cv2, random, sys
  import detectron2.data.transforms as T
  from keras.models import load_model
  sys.path.append(module_dir)
  from utils import index2color, get_predict, correct_pixels

  # Задаем размеры предикта карты сегментации и количества классов
  height = 192
  width = 256
  num_classes = 13

  # Задаем цвета классов
  colors = [[100, 100, 100],
            [0, 0, 100],
            [0, 100, 0],
            [100, 0, 0],
            [0, 100, 100],
            [100, 0, 100],
            [100, 100, 0],
            [200, 200, 200],
            [0, 0, 200],
            [0, 200, 0],
            [200, 0, 0],
            [0, 200, 200],
            [200, 0, 200],
            [200, 200, 0],
            [0, 100, 200],
            [100, 0, 200],
            [0, 0, 0]]

  # Загружаем модели
  model = load_model(module_dir + '12_classes.h5')
  model_for_people = torch.load(module_dir + 'pytorch_model.pt')

  # Загружаем изображение для предикта
  img_for_people = cv2.imread(img_path)

  # Сохраняем размеры оригинального изображения
  original_h = img_for_people.shape[0]
  original_w = img_for_people.shape[1]

  # Препроцессинг для модели 12 классов
  img = cv2.resize(img_for_people, (width, height), interpolation=cv2.INTER_AREA)
  img = np.array(img)[:,:,::-1] / 255
  # Получаем предикт
  pred, people_aura = get_predict(model, model_for_people, img, img_for_people, height, width)
  # Исправляем некоторые пиксели
  pred = correct_pixels(pred, people_aura, num_classes)
  # Переводим индексы классов в цвета
  pred = index2color(pred, num_classes, colors)
  # Возвращаем оригинальные размеры
  pred = cv2.resize(pred, (original_w, original_h), interpolation=cv2.INTER_NEAREST)
  # Формируем путь для сохранения изображения
  pred_path = result_dir + img_path[img_path.rindex('/'):img_path.rindex('.')] + '.jpg'
  # Сохраняем изображенияе
  cv2.imwrite(pred_path, np.uint8(pred[:,:,::-1]))

  return pred_path

In [6]:
import os
import time

for image_path in os.listdir(os.path.join(module_dir + 'test_examples/')):
  curr_time = time.time()
  img_path = module_dir + 'test_examples/' + image_path
  predict_dir = predict(img_path, module_dir, result_dir)
  print(predict_dir + f' ready! Time {round(time.time() - curr_time)}')

** fvcore version of PathManager will be deprecated soon. **
** Please migrate to the version in iopath repo. **
https://github.com/facebookresearch/iopath 

** fvcore version of PathManager will be deprecated soon. **
** Please migrate to the version in iopath repo. **
https://github.com/facebookresearch/iopath 



/usr/local/lib/python3.6/dist-packages/detectron2/modeling/roi_heads/fast_rcnn.py:124: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:882.)
  filter_inds = filter_mask.nonzero()


/content/drive/MyDrive/стажировка/12_classes_final/module_detectron2/results//img_10.jpg ready! Time 39
/content/drive/MyDrive/стажировка/12_classes_final/module_detectron2/results//img_9.jpg ready! Time 5
/content/drive/MyDrive/стажировка/12_classes_final/module_detectron2/results//img_8.jpg ready! Time 4
/content/drive/MyDrive/стажировка/12_classes_final/module_detectron2/results//img_7.jpg ready! Time 4
/content/drive/MyDrive/стажировка/12_classes_final/module_detectron2/results//img_6.jpg ready! Time 5
/content/drive/MyDrive/стажировка/12_classes_final/module_detectron2/results//img_5.jpg ready! Time 4
/content/drive/MyDrive/стажировка/12_classes_final/module_detectron2/results//img_4.jpg ready! Time 4
/content/drive/MyDrive/стажировка/12_classes_final/module_detectron2/results//img_1.jpg ready! Time 5
/content/drive/MyDrive/стажировка/12_classes_final/module_detectron2/results//img_2.jpg ready! Time 4
/content/drive/MyDrive/стажировка/12_classes_final/module_detectron2/results//im